In [15]:
from scipy import stats
import numpy as np
import os
import csv
import numpy as np
import scipy.io as sio
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import RFE
from nilearn import connectome
from nilearn import datasets
import shutil

D:\Anaconda\envs\pytorch\lib\site-packages\nilearn\__init__.py:69: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [7]:
np.random.seed(12345678)
#loc:平均值  scale：方差
rvs1 = stats.norm.rvs(loc=5,scale=10,size=500)
rvs2 = stats.norm.rvs(loc=5,scale=10,size=500)
result = stats.ttest_ind(rvs1,rvs2)

In [11]:
result

Ttest_indResult(statistic=0.26833823296238857, pvalue=0.788494433695651)

In [18]:
phenotype = r"Phenotypic_V1_0b_preprocessed1.csv"
def get_subject_score(subject_list, score):
    scores_dict = {}
    with open(phenotype) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if row['SUB_ID'] in subject_list:
                scores_dict[row['SUB_ID']] = row[score]
    return scores_dict

def Load_Raw_Data(atlas="aal"):

    subject_IDs = np.genfromtxt(r"subject_IDs.txt", dtype=str)
    labels = get_subject_score(subject_IDs, score='DX_GROUP')
    # adj = Reader.create_affinity_graph_from_scores(['SEX', 'SITE_ID'], subject_IDs)
    num_nodes = len(subject_IDs)
    y = np.zeros([num_nodes])
    for i in range(num_nodes):
        y[i] = int(labels[subject_IDs[i]])
    Label = y - 1
    Raw_Features = np.load(rf"{atlas}_871.npy")
    return Raw_Features, Label

In [43]:
features, label = Load_Raw_Data('dosenbach160')
features.shape, label.shape

((871, 12880), (871,))

In [44]:
# 获取正常人和自闭症样本的索引
hc = (label==1)
asd = (label==0)
asd

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [45]:
# 循环遍历每列特征, 计算类别之间的 p-value, 取小的前 K 个特征。
pvalue_list = []
from tqdm import tqdm
for i in tqdm(range(features.shape[1])):
    hc_data = features[:,i][hc]
    asd_data = features[:,i][asd]
    pvalue = stats.ttest_ind(hc_data,asd_data)[1]
    pvalue_list.append(pvalue)
pvalue_list # 所有特征在两类别之间的显著性检验

100%|██████████| 12880/12880 [00:04<00:00, 2773.75it/s]


[0.5385232139764137,
 0.20137198504150045,
 0.03330489146479833,
 0.23407943455604682,
 0.040471002003973924,
 0.0006578947009027804,
 0.4805758918685964,
 0.5941010049091544,
 0.9055795524189709,
 0.107709155482061,
 0.6918373035992968,
 0.0019053289943579024,
 0.0016270840297229032,
 0.13337711678035408,
 0.720959366435094,
 0.0004920977894259959,
 0.5315040208947621,
 0.44317416788028474,
 0.025102706773643207,
 0.2522329217147806,
 0.2727213929662734,
 0.1816685876825668,
 0.9489107371149744,
 0.5040143837125163,
 0.6614384486426885,
 0.8876272418574638,
 0.4793955712975938,
 0.009830902929935671,
 0.4498479206104512,
 0.7091958914267327,
 0.023388560033648993,
 0.41918992750384565,
 0.9465713916691367,
 0.7099078108710912,
 0.6652824547767415,
 0.6016004233761172,
 0.6638872124760876,
 0.28989303374586756,
 0.38417918133689,
 0.9180874894094413,
 0.9368365331020148,
 0.7105981396354826,
 0.4283613900390274,
 0.8518162877280223,
 0.08619401035430958,
 0.14629258407227103,
 0.348748

In [46]:
sort_index = np.argsort(pvalue_list) # 从小到大排序
sort_index

array([1042,  954,  103, ...,  254,   73, 4958], dtype=int64)

In [47]:
pvalue_list[sort_index[0]]

1.3951101543583635e-05

In [48]:
k = 512 # 筛选512维特征
feat_test = features[:,sort_index[:k]]
feat_test

array([[ 4.89872012e-01,  1.21092232e-01,  5.51401829e-01, ...,
         2.15601721e-01,  5.25516517e-01,  3.61123202e-02],
       [ 4.18072947e-01,  1.99389885e-01,  2.79319305e-01, ...,
         7.25937370e-02,  5.81017345e-02,  3.28840264e-01],
       [ 3.35084651e-01,  1.42725994e-01,  6.52819816e-01, ...,
         2.58279899e-01,  4.16818851e-01,  7.22412099e-06],
       ...,
       [ 3.05452415e-01,  2.60981799e-01,  1.45255288e-01, ...,
         6.67766145e-01, -2.06021857e-01,  2.62128195e-01],
       [ 1.95731829e-01,  1.06882406e-01,  5.89623596e-02, ...,
         2.60962590e-02,  1.43808262e-01,  1.02238945e-01],
       [ 6.06812826e-01,  3.66912871e-01,  8.33820604e-01, ...,
         3.01195487e-01,  1.67180333e-02,  2.71638283e-01]])

In [49]:
np.save('dosenbach160_871_Test.npy', feat_test)